In [1]:

from alpaca.data.requests import CryptoBarsRequest, StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timedelta
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.trading.client import TradingClient
import pandas as pd
import yaml
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetClass, AssetStatus
import pickle

In [2]:
keys = yaml.safe_load(open('../keys.yaml', 'r'))

In [3]:
stock_client = StockHistoricalDataClient(keys['paper_key'], keys['paper_secret'])
crypto_client = CryptoHistoricalDataClient()
trading_client = TradingClient(keys['paper_key'], keys['paper_secret'])

In [4]:
search_params = GetAssetsRequest(asset_class = 'crypto', status = 'active')
crypto_assets = trading_client.get_all_assets(search_params)
crypto_symbols = [dict(s)['symbol'] for s in crypto_assets]
crypto_symbols

['BTC/USD',
 'BTC/USDC',
 'BTC/USDT',
 'CRV/USD',
 'CRV/USDC',
 'DOGE/USD',
 'DOGE/USDT',
 'DOT/USD',
 'DOT/USDC',
 'ETH/BTC',
 'ETH/USD',
 'ETH/USDC',
 'ETH/USDT',
 'GRT/USD',
 'GRT/USDC',
 'LINK/BTC',
 'LINK/USD',
 'LINK/USDC',
 'LINK/USDT',
 'LTC/BTC',
 'LTC/USD',
 'LTC/USDC',
 'LTC/USDT',
 'MKR/USD',
 'MKR/USDC',
 'SHIB/USD',
 'SHIB/USDC',
 'SHIB/USDT',
 'SUSHI/USD',
 'SUSHI/USDC',
 'SUSHI/USDT',
 'UNI/BTC',
 'UNI/USD',
 'UNI/USDC',
 'UNI/USDT',
 'USDT/USD',
 'USDT/USDC',
 'XTZ/USD',
 'XTZ/USDC',
 'YFI/USD',
 'YFI/USDC',
 'YFI/USDT',
 'DOGE/USDC',
 'USDC/USD',
 'AAVE/USD',
 'AAVE/USDC',
 'AAVE/USDT',
 'AVAX/USD',
 'AVAX/USDC',
 'AVAX/USDT',
 'BAT/USD',
 'BAT/USDC',
 'BCH/BTC',
 'BCH/USD',
 'BCH/USDC',
 'BCH/USDT']

In [5]:
search_params = GetAssetsRequest(asset_class = 'us_equity', status = 'active')
stock_assets = trading_client.get_all_assets(search_params)
stock_symbols = [dict(s)['symbol'] for s in stock_assets]
len(stock_symbols)

11698

In [6]:
dfs = []
chunksize = 1000
chunks = [stock_symbols[i:i + chunksize] for i in range(0, len(stock_symbols), chunksize)]
for chunk in chunks:
    request_params = StockBarsRequest(
        symbol_or_symbols=chunk,
        timeframe=TimeFrame.Day,
        start=datetime(2024, 4, 8),
        end= datetime(2024, 4, 13)
    )
    stock_bars = stock_client.get_stock_bars(request_params=request_params)
    df = stock_bars.df
    dfs.append(df)
    
df_whole = pd.concat(dfs, axis = 0)
df_whole.shape

(55455, 7)

In [7]:
df_whole.head(20)


open      high       low     close  \
symbol timestamp                                                           
AAPL   2024-04-08 04:00:00+00:00  169.0300  169.2000  168.2400  168.4500   
       2024-04-09 04:00:00+00:00  168.7000  170.0800  168.3500  169.6700   
       2024-04-10 04:00:00+00:00  168.8000  169.0900  167.1100  167.7800   
       2024-04-11 04:00:00+00:00  168.3400  175.4600  168.1600  175.0400   
       2024-04-12 04:00:00+00:00  174.2600  178.3600  174.2100  176.5500   
ACEL   2024-04-08 04:00:00+00:00   11.7200   11.7700   11.6910   11.7100   
       2024-04-09 04:00:00+00:00   11.6700   11.7500   11.6300   11.6600   
       2024-04-10 04:00:00+00:00   11.4300   11.5100   11.0700   11.1300   
       2024-04-11 04:00:00+00:00   11.1500   11.2200   11.0000   11.2200   
       2024-04-12 04:00:00+00:00   11.1200   11.1700   10.9800   11.0100   
ACSI   2024-04-08 04:00:00+00:00   51.0677   51.0677   51.0677   51.0677   
       2024-04-09 04:00:00+00:00   51.1536   51.1536   51.1536   51.1536   
       2024-04-10 04:00:00+00:00   50.6600   50.6600   50.6600   50.6600   
       2024-04-11 04:00:00+00:00   50.9830   50.9830   50.9830   50.9830   
       2024-04-12 04:00:00+00:00   50.1771   50.1771   50.1771   50.1771   
ALDX   2024-04-08 04:00:00+00:00    4.9500    4.9550    4.2950    4.3800   
       2024-04-09 04:00:00+00:00    4.5500    4.6800    4.3300    4.4300   
       2024-04-10 04:00:00+00:00    4.3000    4.3000    4.0950    4.2500   
       2024-04-11 04:00:00+00:00    4.3000    4.4800    4.2200    4.4000   
       2024-04-12 04:00:00+00:00    4.3600    4.4700    4.1900    4.2500   

                                       volume  trade_count        vwap  
symbol timestamp                                                        
AAPL   2024-04-08 04:00:00+00:00   37425513.0     549987.0  168.663679  
       2024-04-09 04:00:00+00:00   42451209.0     541699.0  169.156578  
       2024-04-10 04:00:00+00:00   49709336.0     647589.0  167.991324  
       2024-04-11 04:00:00+00:00   91070275.0     828410.0  172.695893  
       2024-04-12 04:00:00+00:00  101617429.0     959162.0  176.341741  
ACEL   2024-04-08 04:00:00+00:00     133755.0       2284.0   11.719700  
       2024-04-09 04:00:00+00:00     158262.0       2390.0   11.673397  
       2024-04-10 04:00:00+00:00     315823.0       4056.0   11.209101  
       2024-04-11 04:00:00+00:00     247294.0       4077.0   11.158861  
       2024-04-12 04:00:00+00:00     221395.0       3317.0   11.033442  
ACSI   2024-04-08 04:00:00+00:00        190.0         51.0   51.170039  
       2024-04-09 04:00:00+00:00        191.0         30.0   51.101388  
       2024-04-10 04:00:00+00:00        198.0         33.0   50.613207  
       2024-04-11 04:00:00+00:00        161.0         42.0   50.868385  
       2024-04-12 04:00:00+00:00         20.0         20.0   50.196005  
ALDX   2024-04-08 04:00:00+00:00    1041406.0       7318.0    4.506682  
       2024-04-09 04:00:00+00:00    1050308.0       6860.0    4.460414  
       2024-04-10 04:00:00+00:00     643383.0       4975.0    4.204877  
       2024-04-11 04:00:00+00:00     589744.0       4789.0    4.369972  
       2024-04-12 04:00:00+00:00     582033.0       4891.0    4.293153

In [10]:
df_g = df_whole.groupby(level = 'symbol')['trade_count'].mean().sort_values(ascending = False)
df_g

symbol
TSLA     979573.4
NVDA     866760.4
AAPL     705369.4
SPY      616073.6
SOXL     551379.0
           ...   
RFAC          0.0
RFACU         0.0
ASCBU         0.0
AIMAU         0.0
CMRA          0.0
Name: trade_count, Length: 11101, dtype: float64

In [9]:
top_symbols = df_g.head(500).index.tolist()
with open('../data/top500_symbols.pickle', 'wb') as file:
    pickle.dump(top_symbols, file)